# This notebook is there to replicate the results of G&W 2022


In [2]:
import pandas as pd
import numpy as np

# ---------------------------------------------------------
# 1. LOAD AND PREP DATA
# ---------------------------------------------------------

# Replace this with your actual file name / path:
# df_raw = pd.read_excel("GW_monthly_data.xlsx", sheet_name="DATA")
# or if you already have the DataFrame, just set:
# df_raw = your_dataframe
def load_data(file_path="../Data/GoyalAndWelch.xlsx"):
    """Load data from an Excel file."""
    return pd.read_excel(file_path, sheet_name="Monthly")

df_raw = load_data()

# --- basic cleaning

# The GW files usually have an integer YYYYMM column called 'date'
# If yours is already a DateTimeIndex, you can skip this.
def format_date(df):
    """Format the date column to datetime."""
    df['date'] = pd.to_datetime(df['yyyymm'], format='%Y%m', errors='coerce')
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index('date').sort_index()
    df.drop(columns=['yyyymm'], inplace=True)
    return df
df_raw = format_date(df_raw)

# Equity premium: GW define it as market return (incl. dividends) minus riskfree. :contentReference[oaicite:0]{index=0}
# Adapt the column names to your file (often 'ret' and 'Rfree').
df_raw['eq_prem'] = df_raw['ret'] - df_raw['Rfree']



/opt/anaconda3/envs/tsfm/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [11]:
# ---------------------------------------------------------
# 2. GENERIC FUNCTION TO COMPUTE OOS R2
#    (Goyal–Welch-style, expanding window, 1-month ahead)
# ---------------------------------------------------------

def oos_r2_gw(
    y, x,
    initial_years=20,         # first 20 years = in-sample only
    freq_per_year=12,        # monthly
    ct_trunc=False,          # Campbell–Thompson truncation at 0
    ct_sign=None             # +1 or -1 if you want to impose slope sign, else None
):
    """
    y: equity premium series (numpy array or pandas Series)
    x: predictor at same dates as y
       The forecast is y_{t+1} using x_t, so we lag x by one.

    Returns:
        R2_oos : standard OOS R^2
        R2_oos_ct : if ct_trunc/sign used, the OOSCT R^2; else None
        preds : conditional forecasts (aligned with OOS part of y)
        preds_ct : CT-variant forecasts (if requested)
    """

    # Drop any rows with missing y or x
    mask = (~pd.isna(y)) & (~pd.isna(x))
    y = np.asarray(y[mask], dtype=float)
    x = np.asarray(x[mask], dtype=float)

    # Align: y_{t+1} on x_t  → lose first y and last x
    y_al = y[1:]
    x_al = x[:-1]

    T = len(y_al)
    init_obs = initial_years * freq_per_year    # e.g. 20*12 = 240
    if T <= init_obs:
        raise ValueError("Not enough data for the requested initial window.")

    # storage for errors and forecasts
    e_model = []
    e_mean = []
    preds = []
    preds_ct = [] if (ct_trunc or ct_sign is not None) else None

    # rolling (expanding window) OLS forecasts
    for t in range(init_obs, T):
        # estimation sample 0 .. t-1
        y_est = y_al[:t]
        x_est = x_al[:t]

        # OLS: y = a + b x
        X = np.column_stack([np.ones_like(x_est), x_est])
        beta_hat = np.linalg.lstsq(X, y_est, rcond=None)[0]
        a_hat, b_hat = beta_hat

        # historical mean model (unconditional)
        mu_hat = y_est.mean()

        # 1-step-ahead forecast for y_t, using x_t
        x_t = x_al[t]
        y_hat = a_hat + b_hat * x_t

        # store normal-GW errors
        y_realized = y_al[t]
        e_model.append(y_realized - y_hat)
        e_mean.append(y_realized - mu_hat)
        preds.append(y_hat)

        # Campbell–Thompson tweaks, if requested
        if preds_ct is not None:
            y_hat_ct = y_hat

            # (1) sign restriction on slope (if theory says sign)
            if ct_sign is not None and np.sign(b_hat) != np.sign(ct_sign):
                # use unconditional mean instead of "perverse" slope
                y_hat_ct = mu_hat

            # (2) truncate negative forecasts of the equity premium at 0 :contentReference[oaicite:1]{index=1}
            if ct_trunc and y_hat_ct < 0:
                y_hat_ct = 0.0

            preds_ct.append(y_hat_ct)

    e_model = np.asarray(e_model)
    e_mean = np.asarray(e_mean)
    preds = np.asarray(preds)

    # GW OOS R^2 (Eq. (6): 1 - MSE_model / MSE_mean). :contentReference[oaicite:2]{index=2}
    mse_model = np.mean(e_model**2)
    mse_mean = np.mean(e_mean**2)
    r2_oos = 1.0 - mse_model / mse_mean

    r2_oos_ct = None
    if preds_ct is not None:
        preds_ct = np.asarray(preds_ct)
        e_model_ct = y_al[init_obs:] - preds_ct
        mse_model_ct = np.mean(e_model_ct**2)
        r2_oos_ct = 1.0 - mse_model_ct / mse_mean

    return r2_oos, r2_oos_ct, preds, preds_ct


# ---------------------------------------------------------
# 3. EXAMPLE: REPLICATE OOS R2 FOR A MONTHLY VARIABLE
# ---------------------------------------------------------

# choose your predictor name here, e.g.:
#   'dp'  → dividend-price ratio
#   'tbl' → T-bill rate
#   'tms' → term spread
predictor_name = 'd/p'   # change to whatever column you want

# restrict to the same sample as Table 3 in GW (Dec 1927 – Dec 2004). :contentReference[oaicite:3]{index=3}
# Adjust these if you want the 2024 extended sample instead.
sample = df_raw.loc['1927-12':'2004-12']

y = sample['eq_prem']
x = sample[predictor_name]

# Plain GW-style OOS R^2:
r2_oos, r2_oos_ct, preds, preds_ct = oos_r2_gw(
    y, x,
    initial_years=20,
    freq_per_year=12,
    ct_trunc=False,   # set True for CT truncation
    ct_sign=None      # e.g. +1 for variables that "should" have positive slope
)

print(f"OOS R^2 for {predictor_name}: {r2_oos:.4f}")

# If you also want the Campbell–Thompson OOSCT R^2
r2_oos_ct_only, r2_oos_ct, _, _ = oos_r2_gw(
    y, x,
    initial_years=20,
    freq_per_year=12,
    ct_trunc=True,     # truncate negative forecasts at 0
    ct_sign=None       # or +1 / -1 if you want the slope-sign rule
)

print(f"OOSCT R^2 (CT) for {predictor_name}: {r2_oos_ct:.4f}")


OOS R^2 for d/p: 0.0006
OOSCT R^2 (CT) for d/p: 0.0027


In [ ]:
import pandas as pd
import numpy as np

# Load your monthly GW data
# df = pd.read_excel("monthly_2005.xlsx", sheet_name="Data")  # example

df = load_data()

df['date'] = pd.to_datetime(df['yyyymm'], format='%Y%m', errors='coerce')
df['date'] = pd.to_datetime(df['date'])
# Make a PeriodIndex from the integer date YYYYMM column
df['date'] = pd.PeriodIndex(df['date'].astype(str), freq='M')
df = df.set_index('date').sort_index()

# Equity premium: log total return on market minus log risk-free rate
# GW define returns as log(1+simple_return) and then multiply by 100 (percent),
# but the scale cancels out in R², so we only need consistency.
df['log_ret_mkt']   = np.log1p(df['ret'])            # total market simple return
df['log_rf']        = np.log1p(df['Rfree'])          # T-bill simple return
df['rp_div_month']  = df['log_ret_mkt'] - df['log_rf']

# Dividend-price ratio d/p (log(D12) − log(price))
df['dp'] = np.log(df['d12']) - np.log(df['price'])


/opt/anaconda3/envs/tsfm/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [20]:
import numpy as np
import pandas as pd

def get_statistics(
    ts_df,
    indep,
    dep_IS,
    dep_OOS=None,
    h=1,
    start=None,
    end=None,
    est_periods_OOS=20,
):
    """
    Python translation of the R function in the blog.

    Parameters
    ----------
    ts_df : DataFrame
        Full time series (like the 'ts_annual' or 'ts_monthly' in the blog)
        indexed by a Date/PeriodIndex, sorted.
    indep : str
        Name of predictor column (e.g. 'dp').
    dep_IS : str
        Column name of the dependent variable for the IS regression
        (e.g. 'erp_log' for monthly Table 3's first column).
    dep_OOS : str or None
        Dependent variable for OOS forecasts.
        - If None, use dep_IS also for OOS (like the original R code).
        - For monthly GW/CT table, use 'erp_simple' here.
    h : int
        Overlap degree (kept for compatibility; not used for h=1).
    start, end : like '1927-01', '2004-12'
        Bounds of the estimation sample; if None, use full ts_df.
    est_periods_OOS : int
        Number of initial periods used only for IS (e.g. 20 years).

    Returns
    -------
    dict with IS_R2, IS_aR2, OOS_R2, OOS_oR2 and the error vectors.
    """

    if dep_OOS is None:
        dep_OOS = dep_IS

    # --- 1. Subset window like window(ts_df, start, end) in R ---
    if start is not None:
        ts_df = ts_df[ts_df.index >= start]
    if end is not None:
        ts_df = ts_df[ts_df.index <= end]

    # keep only relevant columns
    df = ts_df[[indep, dep_IS, dep_OOS]].dropna()

    # ---------- IN-SAMPLE: dep_IS on lagged indep ----------
    y_IS = df[dep_IS].iloc[1:].to_numpy()
    x_IS = df[indep].iloc[:-1].to_numpy()
    X_IS = np.column_stack([np.ones_like(x_IS), x_IS])

    beta_IS, _, _, _ = np.linalg.lstsq(X_IS, y_IS, rcond=None)
    a_IS, b_IS = beta_IS
    y_hat_IS = X_IS @ beta_IS
    resid_IS = y_IS - y_hat_IS

    sse = np.sum(resid_IS ** 2)
    sst = np.sum((y_IS - y_IS.mean()) ** 2)
    IS_R2 = 1.0 - sse / sst

    n_IS = len(y_IS)
    k = 1  # one regressor
    IS_aR2 = 1.0 - (1.0 - IS_R2) * (n_IS - 1) / (n_IS - k - 1)

    # ---------- OOS: use dep_OOS, expanding window ----------
    values = df[[indep, dep_OOS]].to_numpy()
    n = len(values)

    # est_periods_OOS is in *periods* (years in R, but you choose!)
    # For monthly, you’d pass est_periods_OOS = 20*12.
    est_obs = est_periods_OOS

    if n <= est_obs + 1:
        raise ValueError("Not enough observations for chosen OOS window.")

    OOS_error_N = []
    OOS_error_A = []

    for i in range(est_obs, n - 1):
        # estimation window: 0..i  (inclusive)
        sub = values[:i + 1, :]
        x_sub = sub[:, 0]
        y_sub = sub[:, 1]

        # y_t on x_{t-1}, same lagging as in IS
        y_est = y_sub[1:]
        x_est = x_sub[:-1]
        X_est = np.column_stack([np.ones_like(x_est), x_est])

        beta, _, _, _ = np.linalg.lstsq(X_est, y_est, rcond=None)
        a_hat, b_hat = beta

        # actual ERP at time t+1 (same dep_OOS series)
        actual = values[i + 1, 1]
        # predictor at time t
        x_t = values[i, 0]

        # 1) historical mean model (mean of y_est, up to time i)
        mu_hat = y_est.mean()
        err_N = actual - mu_hat                # like R: actual - mean

        # 2) regression model
        pred = a_hat + b_hat * x_t
        err_A = pred - actual                  # like R: pred - actual

        OOS_error_N.append(err_N)
        OOS_error_A.append(err_A)

    OOS_error_N = np.asarray(OOS_error_N)
    OOS_error_A = np.asarray(OOS_error_A)

    MSE_N = np.mean(OOS_error_N ** 2)
    MSE_A = np.mean(OOS_error_A ** 2)
    OOS_R2 = 1.0 - MSE_A / MSE_N

    # --- Adjusted OOS R² exactly as in the R code ---
    # T is length(!is.na(ts_df[, dep_OOS])) over the full original series
    T_full = ts_df[dep_OOS].notna().sum()
    df_resid = n_IS - (k + 1)   # reg$df.residual from IS regression

    OOS_oR2 = OOS_R2 - (1.0 - OOS_R2) * df_resid / (T_full - 1)

    return {
        "IS_R2": IS_R2,
        "IS_aR2": IS_aR2,
        "OOS_R2": OOS_R2,
        "OOS_oR2": OOS_oR2,
        "IS_residuals": resid_IS,
        "OOS_error_N": OOS_error_N,
        "OOS_error_A": OOS_error_A,
    }





In [22]:
# 1. Build predictors and returns
df_m = df.copy()
df_m['erp_simple'] = df_m['ret'] - df_m['Rfree']
df_m['erp_log']    = np.log1p(df_m['ret']) - np.log1p(df_m['Rfree'])

# 2. Use ORIGINAL 2005 data and sample 1927-01 to 2004-12 if you want
#    to be as close as possible to Table 3.
stats_dp = get_statistics(
    ts_df=df_m,
    indep='d/p',            # or whatever the exact column name is
    dep_IS='erp_log',       # log equity premium for IS column
    dep_OOS='erp_simple',   # simple equity premium for OOS columns
    start='1927-01',
    end='2004-12',
    est_periods_OOS=20 * 12 # 20 years of monthly data
)

print("IS R² (log, d/p):      ", stats_dp["IS_R2"])
print("Adj. IS R² (log, d/p): ", stats_dp["IS_aR2"])
print("OOS R² (simple, d/p):  ", stats_dp["OOS_R2"])
print("Adj. OOS R² (simple):  ", stats_dp["OOS_oR2"])


IS R² (log, d/p):       0.00368546262431102
Adj. IS R² (log, d/p):  0.0026176013838226098
OOS R² (simple, d/p):   0.0028078455035774885
Adj. OOS R² (simple):   -0.9922512776463287


In [16]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from plotnine import (
    ggplot,
    aes,
    geom_line,
    geom_rect,
    scale_y_continuous,
    scale_x_continuous,
)

# --- 1. Load Libraries and Data ---

# Note: Adjust file paths as needed.
# R's read.csv2 uses ';' as a separator and ',' as a decimal.

monthly = pd.read_excel("../Data/GoyalAndWelch.xlsx", sheet_name="Monthly")
annual = pd.read_excel("../Data/GoyalAndWelch.xlsx", sheet_name="Annual")


# Convert monthly date column
monthly['Datum'] = pd.to_datetime(monthly['yyyymm'], format='%Y%m', errors='coerce')
annual['Datum'] = pd.to_datetime(annual['yyyy'], format='%Y', errors='coerce')



/opt/anaconda3/envs/tsfm/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
/opt/anaconda3/envs/tsfm/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored


In [17]:
# --- 2. Data Preparation (Feature Engineering) ---
# This replicates the data.table 'annual[, ... := ...]' operations.

annual['IndexDiv'] = annual['price'] + annual['d12']
annual['dp'] = np.log(annual['d12']) - np.log(annual['price'])
annual['ep'] = np.log(annual['e12']) - np.log(annual['price'])

# Calculate vec_dy
# R: c(NA, annual[2:nrow(annual), log(D12)] - annual[1:(nrow(annual)-1), log(Index)])
log_d12 = np.log(annual['d12'].iloc[1:].values)
log_index_prev = np.log(annual['price'].iloc[:-1].values)
vec_dy = np.concatenate([[np.nan], log_d12 - log_index_prev])
annual['dy'] = vec_dy

# Calculate logret
# R: c(NA,diff(log(Index))) -> pandas .diff() is cleaner
annual['logret'] = np.log(annual['price']).diff()

# Calculate vec_logretdiv
# R: vec_logretdiv <- c(NA, log(annual[2:nrow(annual), IndexDiv]/annual[1:(nrow(annual)-1), Index]))
# Note: The R code defines this variable twice; we use the second, overwriting definition.
log_ret_div_val = np.log(annual['IndexDiv'].iloc[1:].values / annual['price'].iloc[:-1].values)
vec_logretdiv = np.concatenate([[np.nan], log_ret_div_val])
annual['logretdiv'] = vec_logretdiv

annual['logRfree'] = np.log(annual['Rfree'] + 1)
annual['rp_div'] = annual['logretdiv'] - annual['logRfree']





monthly['IndexDiv'] = monthly['price'] + monthly['d12']
monthly['dp'] = np.log(monthly['d12']) - np.log(monthly['price'])
monthly['ep'] = np.log(monthly['e12']) - np.log(monthly['price'])   

log_d12 = np.log(monthly['d12'].iloc[1:].values)
log_index_prev = np.log(monthly['price'].iloc[:-1].values)
vec_logretdiv = np.concatenate([[np.nan], log_d12 - log_index_prev])
monthly['dy'] = vec_logretdiv

monthly['logret'] = np.log(monthly['price']).diff()
log_ret_div_val = np.log(monthly['IndexDiv'].iloc[1:].values / monthly['price'].iloc[:-1].values)
vec_logretdiv = np.concatenate([[np.nan], log_ret_div_val])
monthly['logretdiv'] = vec_logretdiv

monthly['logRfree'] = np.log(monthly['Rfree'] + 1)
monthly['rp_div'] = monthly['logretdiv'] - monthly['logRfree']




In [18]:
# --- 3. Set up Time Series Index ---
# This is the Python equivalent of R's ts() object and window() function.
# We set the 'Datum' (year) as the index to easily select date ranges.
if 'Datum' in annual.columns:
    annual = annual.set_index('Datum')
else:
    print("Warning: 'Datum' column not found. Assuming a simple range index.")

# The R code plots ts_annual here. We'll skip that as the main work is in the function.


if 'Datum' in monthly.columns:
    monthly = monthly.set_index('Datum')
else:
    print("Warning: 'Datum' column not found. Assuming a simple range index.")

In [15]:
annual

,yyyy,price,d12,e12,ret,retx,AAA,BAA,lty,ltr,...,shtint,disag,IndexDiv,dp,ep,dy,logret,logretdiv,logRfree,rp_div
Datum,,,,,,,,,,,,,,,,,,,,,
1871-01-01,1871,4.740000,0.260000,0.40,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.000000,-2.903111,-2.472328,NaN,NaN,NaN,0.048419,NaN
1872-01-01,1872,5.070000,0.300000,0.43,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.370000,-2.827314,-2.467311,-2.760010,0.067304,0.124791,0.071515,0.053276
1873-01-01,1873,4.420000,0.330000,0.46,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.750000,-2.594802,-2.262668,-2.732003,-0.137201,-0.065196,0.087750,-0.152946
1874-01-01,1874,4.540000,0.330000,0.46,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.870000,-2.621590,-2.289456,-2.594802,0.026787,0.096954,0.056677,0.040277
1875-01-01,1875,4.370000,0.300000,0.36,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.670000,-2.678736,-2.496414,-2.716900,-0.038164,0.028232,0.044875,-0.016643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-01,2020,3756.070000,58.278846,94.13,0.193039,0.171269,0.0226,0.0316,0.0093,0.166612,...,-1.481539,6.401161,3814.348846,-4.165889,-3.686452,-4.015240,0.150650,0.166046,0.005242,0.160804
2021-01-01,2021,4766.180000,60.397117,197.87,0.284753,0.266830,0.0265,0.0330,0.0147,-0.054275,...,-1.886112,4.601180,4826.577117,-4.368359,-3.181690,-4.130187,0.238172,0.250764,0.000500,0.250264
2022-01-01,2022,3839.500000,66.922828,172.75,-0.181886,-0.195288,0.0443,0.0559,0.0362,-0.124626,...,-2.016237,2.884048,3906.422828,-4.049557,-3.101252,-4.265760,-0.216203,-0.198923,0.012762,-0.211685


In [19]:
# --- 4. Define Statistics Function ---

def get_statistics(ts_df, indep, dep, h=1, start='1872-01-01', end='2006-01-01', est_periods_OOS=20, freq = 'YS'):
    """
    Performs In-Sample (IS) and Out-of-Sample (OOS) predictability analysis.
    
    Translated from the R function provided.
    """
    
    #### IS ANALYSIS ####
    
    # 1. Historical mean model
    ts_window_is = ts_df.loc[start:end]
    avg = ts_window_is[dep].mean()
    IS_error_N = ts_window_is[dep] - avg
    
    # 2. OLS model
    # R: dyn$lm(eval(parse(text=dep)) ~ lag(eval(parse(text=indep)), -1), ...)
    # Python: Use pandas.shift() for the lag
    
    df_reg = ts_window_is[[dep, indep]].copy()
    df_reg[f'{indep}_lag1'] = df_reg[indep].shift(1)
    
    # Drop NAs from lagging
    df_reg = df_reg.dropna()
    
    Y = df_reg[dep]
    X = sm.add_constant(df_reg[f'{indep}_lag1'])
    
    reg = sm.OLS(Y, X).fit()
    IS_error_A = reg.resid

    #### OOS ANALYSIS ####
    OOS_error_N = []
    OOS_error_A = []
    
    # R loop: (start + est_periods_OOS):(end-1)
    # Python range goes up to, but does not include, the end value.
    start_dt = pd.to_datetime(start)
    end_dt = pd.to_datetime(end)

    # Add 20 years to start
    start_oos = start_dt + pd.DateOffset(years=est_periods_OOS)

    # Now you can make a date range, resample, etc.
    date_range = pd.date_range(start=start_oos, end=end_dt, freq=freq)
    for i in date_range:
        # Get the actual value you want to predict (for time t+1)
        actual_ERP = ts_df.loc[i + pd.DateOffset(years=1), dep]
        
        # 1. Historical mean model
        # Use all data available *up to* time t (inclusive)
        mean_val = ts_df.loc[start:i, dep].mean()
        OOS_error_N.append(actual_ERP - mean_val)
        
        # 2. OLS model
        # R: dyn$lm(..., data=window(ts_df, start, i))
        reg_data = ts_df.loc[start:i, [dep, indep]].copy()
        reg_data[f'{indep}_lag1'] = reg_data[indep].shift(1)
        
        # Fit OLS on expanding window
        Y_oos = reg_data[dep]
        X_oos = sm.add_constant(reg_data[f'{indep}_lag1'])
        
        # Use missing='drop' to handle the NA from shift()
        reg_OOS = sm.OLS(Y_oos, X_oos, missing='drop').fit()
        
        # Compute error
        # Predict ERP for t+1 using data from time t
        # R: newdata=df(x=...window(ts_df, i, i)[, indep]))
        predictor_val = ts_df.loc[i, indep]
        X_pred = [1, predictor_val]  # [constant, lag(indep)]
        
        pred_ERP = reg_OOS.predict(X_pred)[0]
        
        # Note: The R code calculates pred - actual.
        OOS_error_A.append(pred_ERP - actual_ERP)

    # Convert lists to numpy arrays for calculation
    OOS_error_N = np.array(OOS_error_N)
    OOS_error_A = np.array(OOS_error_A)
    
    #### Compute statistics ####
    MSE_N = np.mean(OOS_error_N**2)
    MSE_A = np.mean(OOS_error_A**2)
    T = ts_df[dep].notna().sum()  # Total non-NA observations
    OOS_R2 = 1 - MSE_A / MSE_N
    
    # R: (1-OOS_R2)*(reg$df.residual)/(T - 1)
    OOS_oR2 = OOS_R2 - (1 - OOS_R2) * (reg.df_resid) / (T - 1)
    dRMSE = np.sqrt(MSE_N) - np.sqrt(MSE_A)
    
    #### CREATE PLOT ####
    
    # R: IS <- cumsum(IS_error_N[2:length(IS_error_N)]^2)-cumsum(IS_error_A^2)
    # IS_error_N has 1 more element than IS_error_A (due to lag)
    # So we take IS_error_N[1:] to align them.
    IS = np.cumsum(IS_error_N.iloc[1:].values**2) - np.cumsum(IS_error_A**2)
    OOS = np.cumsum(OOS_error_N**2) - np.cumsum(OOS_error_A**2)
    
    # R: seq.int(from=start + 1 + est_periods_OOS, to=end)
    # x_axis = date_range(pd.to_datetime(start) + pd.DateOffset(years=est_periods_OOS), pd.to_datetime(end) + pd.DateOffset(years=est_periods_OOS))
    
    # # R: IS=IS[(1 + est_periods_OOS):length(IS)]
    # # Python 0-based index:
    # IS_plot = IS[est_periods_OOS:]
    
    # # Create the DataFrame for plotting
    # df = pd.DataFrame({
    #     'x': x_axis,
    #     'IS': IS_plot,
    #     'OOS': OOS
    # })
    
    # # R: df$IS <- df$IS - df$IS[1]
    # df['IS'] = df['IS'] - df['IS'].iloc[0]
    
    # # R: melt(df, id.var="x")
    # df_melted = df.melt('x', var_name='variable', value_name='value')
    
    # Recreate the ggplot object
    # plotGG = (
    #     ggplot(df_melted, aes(x='x', y='value', color='variable'))
    #     + geom_line()
    #     # R: geom_rect(... alpha=0.1)
    #     # Note: In plotnine, we must provide a new df and set inherit_aes=False
    #     + geom_rect(
    #         data=pd.DataFrame({
    #             'xmin': [1973], 'xmax': [1975], 'ymin': [-0.2], 'ymax': [0.2]
    #         }),
    #         aes(xmin='xmin', xmax='xmax', ymin='ymin', ymax='ymax'),
    #         fill='red',
    #         alpha=0.1,
    #         inherit_aes=False
    #     )
    #     + scale_y_continuous('Cumulative SSE Difference', limits=(-0.2, 0.2))
    #     + scale_x_continuous('Year')
    # )
    
    # Return results as a dictionary
    return {
        'IS_error_N': IS_error_N,
        'IS_error_A': IS_error_A,
        'OOS_error_N': OOS_error_N,
        'OOS_error_A': OOS_error_A,
        'IS_R2': reg.rsquared,
        'IS_aR2': reg.rsquared_adj,
        'OOS_R2': OOS_R2,
        'OOS_oR2': OOS_oR2,
        'dRMSE': dRMSE,
        # 'plotGG': plotGG,
        'IS_reg_summary': reg.summary() # Added for convenience
    }



In [ ]:
# --- 5. Run Analyses ---

# Note: The 'ts_annual' object from R is just the 'annual' DataFrame in Python
# To display the plot, you can print it (in a notebook) or save it
#print(dp_stat['plotGG'])
# dp_stat['plotGG'].save('dp_plot.png')

# dy_stat = get_statistics(annual, "dy", "rp_div", start=1872)
# print(dy_stat['plotGG'])

# ep_stat = get_statistics(annual, "ep", "rp_div", start=1872)
# print(ep_stat['plotGG'])

# # You can also inspect other statistics


In [25]:
dp_stat = get_statistics(annual, "d/y", "rp_div", freq = 'YS')

print("\n--- DP Model Statistics ---")
print(f"IS R2: {dp_stat['IS_R2']:.4f}")
print(f"OOS R2: {dp_stat['OOS_R2']:.4f}")
print(f"dRMSE: {dp_stat['dRMSE']:.4f}")


--- DP Model Statistics ---
IS R2: 0.0188
OOS R2: 0.0004
dRMSE: 0.0000


In [24]:
dp_stat = get_statistics(monthly, "d/y", "rp_div", freq = 'MS')
print("\n--- DP Model Statistics ---")
print(f"IS R2: {dp_stat['IS_R2']:.4f}")
print(f"OOS R2: {dp_stat['OOS_R2']:.4f}")
print(f"dRMSE: {dp_stat['dRMSE']:.4f}")


--- DP Model Statistics ---
IS R2: 0.1175
OOS R2: 0.0885
dRMSE: 0.0023
